In [2]:
import networkx as nx
import os
from utils.global_rules import rules

dataDir = "../data/"
cacheDir = "../data/cache/"

读入图数据，G: 无向图，G_directed: 有向图


In [3]:
import importlib

process = importlib.import_module("process")
importlib.reload(process)

G = process.get_graph()

Loading graph...
Initial Node shape:  (2371558, 4)
Initial Link shape:  (3286986, 3)
After drop nan and duplicates:
Node shape:  (2371558, 4)
Link shape:  (3285490, 3)
Undirected graph loaded.


In [23]:
# Create a Directed Graph from the undirected one
G_directed = nx.DiGraph(G)

运行社区检测


In [3]:
import importlib

community_detection = importlib.import_module("utils.community_detection")
importlib.reload(community_detection)

# community_detection.get_community_for_all_groups(G)
community_detection.get_subgraph_for_all_communities(G)

根据以上社区检测算法的结果，选取团伙 [?] 进行分析。


In [5]:
import json
import importlib
import pandas as pd

rules = importlib.import_module("utils.global_rules")
process = importlib.import_module("process")
importlib.reload(rules)
importlib.reload(process)

rules = rules.rules
# group_to_analyze = ["1", "3", "4"]
group_to_analyze = ["1", "2", "3", "4", "5"]
cacheDir = "../data/cache/"
subgraphDir = "../data/subgraph/"
tempDir = "../data/temp/"


def export_subgraph(graph, group_str: str):
    # transform the graph to directed graph
    # graph = graph.to_directed()
    # export the graph to json
    data = nx.readwrite.json_graph.node_link_data(graph)
    # Keep only the nodes and edges
    data = {"nodes": data["nodes"], "edges": data["links"]}
    json.dump(
        data,
        open(tempDir + "group_" + group_str + ".json", "w"),
        indent=4,
    )

    json.dump(
        nx.readwrite.json_graph.node_link_data(graph),
        open(subgraphDir + "group_" + group_str + ".json", "w"),
        indent=4,
    )

    # Create a directory in subgraphDir
    if not os.path.exists(subgraphDir + group_str):
        os.makedirs(subgraphDir + group_str)

    node_data = pd.DataFrame(data["nodes"])
    node_data.to_csv(subgraphDir + group_str + "/nodes.csv", index=False)
    link_data = pd.DataFrame(data["edges"])
    link_data.to_csv(subgraphDir + group_str + "/links.csv", index=False)

    return


def analyse_group(group_str: str):
    print("analyse group: ", group_str)
    # get the evidence
    evidence = rules["evidence"][group_str]

    # # read the group
    # graph = nx.readwrite.json_graph.node_link_graph(
    #     json.load(open(cacheDir + "group_" + group_str + ".json"))
    # )
    # # print graph info
    # print("#nodes: ", graph.number_of_nodes())
    # print("#edges: ", graph.number_of_edges())
    # subgraph = process.get_subgrah(graph, evidence)

    subgraph = process.get_subgraph(G, evidence)
    print("#nodes in subgraph: ", subgraph.number_of_nodes())
    print("#edges in subgraph: ", subgraph.number_of_edges())

    subgraph_filtered = process.filter_subgraph(
        subgraph, countKeepPercent=0.6, pagerankQuantile=0.96
    )
    process.set_core(subgraph_filtered)
    export_subgraph(subgraph_filtered, group_str)

    print()
    return


for group in group_to_analyze:
    analyse_group(group)

# analyse_group("1")

analyse group:  1
#nodes in subgraph:  30767
#edges in subgraph:  82663
Filtering nodes...
Removing nodes by count...
Nodes removed:  9851 Nodes left:  20916
Removing nodes with empty industry and low degree...
Nodes removed:  2114 Nodes left:  18802
Removing nodes by pagerank quantile and degree centrality...
Nodes removed:  17611 Nodes left:  1191
Keeping the max connected component...
Nodes left:  516


KeyError: 'links'